Заполните данные о необходимых переменных:

In [1]:
Today = '31.01.2024'
Mounth = 'Jan'

Group = 'SUEK'

In [2]:
import os
files_list = os.listdir('.')
work_files_list = [x for x in files_list if ('.xlsx' in x and '~' not in x and 'ODGS' in x and '01' in x)] # Месяц
Data_file = [x for x in work_files_list if 'SUEK' in x][0] # Холдинг
Sheet_in_data_file = 'Export Worksheet'

from datetime import date
Output_file = "_".join([str(date.today()), Group, 'SAP', 'ARAP', f'{Mounth}.xlsx'])
Sheet_in_output_file = 'ARAP'

work_files_list

['ODGS 2024 01 M SUEK.xlsx', 'ODGS_2024_01_M_ECh.xlsx']

Открываю следующие файлы:

In [3]:
import pandas as pd
import numpy as np

data_export = pd.read_excel(Data_file, sheet_name = Sheet_in_data_file)

c:\Users\KlimovaAnnaA\AppData\Local\miniconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [4]:
import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\OCP")
from Defs import merge_SalesUnits
from Defs import merge_Mapping
from Defs import Period

In [5]:
data_work = data_export.query('holding == @Group')
# фильтрация по перемнной Group

data_work['org_code'] = merge_Mapping(data_work, col='orgName')
data_work['CompMap'] = merge_SalesUnits(data_work, col='org_code', merge_col='ocpSegment')

data_work['CtyCode'] = merge_Mapping(data_work, col='customerName')
data_work['CtyMap'] = merge_SalesUnits(data_work, col='CtyCode', merge_col='ocpSegment')

data_work['AR_flag'] = data_work['accountPurpose']\
                                            .isin(['Trade receivables', 'Other receivables', 'Provisions'])\
                                            .astype(int)
data_work['AP_flag'] = data_work['accountPurpose']\
                                            .isin(['Trade payables', 'Other payables'])\
                                            .astype(int)

data_work['Amount_USD'] = data_work['presentAmountUSD'].replace({np.NaN : 0})
data_work['Amount_AR_USD'] = data_work['Amount_USD'] * data_work['AR_flag']
data_work['Amount_AP_USD'] = data_work['Amount_USD'] * data_work['AP_flag']

data_work = Period(data_work, day_for_count=Today, col_with_date='dueDate')

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:42: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  day = pd.to_datetime(day_for_count)


Записываю данные в output file:

In [6]:
data_print = data_work[['customerName', 'org_code', 'currencyCode', 'sourceSystem',\
                    'accountPurpose', 'CompMap', 'CtyCode', 'CtyMap','Amount_USD',\
                    'AR_flag', 'Amount_AR_USD', 'AP_flag', 'Amount_AP_USD', 'Days', 'Period']]

Проверка на merge:

In [7]:
check_data = data_work.loc[(data_work['internalExternalEN'] == 'Internal') & (data_work['CtyMap'] == 'External'),\
                                                    ['customerName', 'CtyCode', 'CtyMap']].drop_duplicates()
check_data
# check_data.to_excel('manual_map_ARAP_SUEK.xlsx', index=False)

,customerName,CtyCode,CtyMap
137147,"ООО ""Управление по буровзрывным работам""",External,External
180800,"ООО ""Германий и приложения""",External,External
204231,"АО ""Управляющая компания ""Мурманский транспорт...",External,External
236121,Благотворительный фонд Андрея Мельниченко,External,External
258366,"ООО ""БГТК""",External,External
275548,"АО ""Морцентр-ТЭК""",External,External
317761,SUEK India Pvt Ltd,External,External


In [8]:
check_data2 = data_work.loc[(data_work['internalExternalEN'] == 'Internal') & (data_work['CompMap'] == 'External'),\
                                                    ['orgName', 'org_code','CompMap']].drop_duplicates()
check_data2

,orgName,org_code,CompMap


In [9]:
# assert 1==2

In [10]:
# data_print.to_excel(Output_file, sheet_name = Sheet_in_output_file, index = False)

In [11]:
BS_data = data_work[(data_work.accountPurpose.str.contains('Trade')) | (data_work.accountPurpose.str.contains('Other'))].reset_index(drop=True)
BS_data['Comp_hold'] = merge_SalesUnits(BS_data, col='org_code', merge_col='holding')
BS_data['Cpty_hold'] = merge_SalesUnits(BS_data, col='CtyCode', merge_col='holding')

BS_data = BS_data[(BS_data.Comp_hold == Group) & (BS_data.Cpty_hold == Group)]
BS_data = BS_data[BS_data.CompMap != 'External']
BS_data = BS_data[['CompMap', 'CtyMap', 'Amount_USD']]
BS_pivot = pd.pivot_table(BS_data, values='Amount_USD', index='CompMap', aggfunc='sum', columns='CtyMap',)
BS_pivot

CtyMap,NTC,RUFERT,SGC,SUEK INT,SUEK RU
CompMap,,,,,
NTC,-627357.89,-1201963.75,4.374756e+04,4.354061e+06,-6.295659e+07
RUFERT,307281.07,0.02,-4.303268e+04,NaN,3.140277e+05
SGC,108772.17,43032.72,3.931081e+06,NaN,-1.331444e+08
SUEK INT,-2442357.47,NaN,NaN,1.151956e+06,-9.804473e+08
SUEK RU,26483424.04,-308381.77,1.226782e+08,7.413301e+08,4.327851e+06


In [12]:
# with pd.ExcelWriter(Output_file, engine='openpyxl', mode='a') as writer:
#     BS_pivot.to_excel(writer,sheet_name='BS_to_BS')

In [13]:
assert 1==2

AssertionError: 

In [ ]:
BS_pivot.to_excel(Output_file, sheet_name = 'BS_to_BS')
with pd.ExcelWriter(Output_file, engine='openpyxl', mode='a') as writer:
    data_print.to_excel(writer,sheet_name=Sheet_in_output_file, index=False)
